# Mid-gestation fetal cortex dataset: Cluster identification and characterization

__Upstream Steps__

* QC filter on cells
* Expression filter on genes
* Normalization and log10 transformation by Scanpy
* HVG by Triku
* Integration by Harmony
* Dimensionality reduction after integration
* Cluster identification
* Cluster characterization

__This notebook__

* Define pseudo-ordering and identify the correct way to do it



----

# 1. Environment Set Up

## 1.1 Library upload

In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import palantir
from datetime import datetime

from gprofiler import GProfiler

In [ ]:
sc.settings.verbosity = 3
sc.settings.set_figure_params(dpi=80)

## 1.2 Start Computation time

In [ ]:
print(datetime.now())

## 1.3 Result file

In [ ]:
#results_file = '/group/brainomics/Intermediate/6_AdataDPT_intermediate.h5ad'

----

# 2. Read input files  

In [ ]:
adata = sc.read('/group/brainomics/Intermediate/5_AdataDPT.h5ad')

In [ ]:
print('Loaded Normalizes AnnData object: number of cells', adata.n_obs)
print('Loaded Normalizes AnnData object: number of genes', adata.n_vars)

print('Available metadata for each cell: ', adata.obs.columns)

----

# 3. Palantir

Identify in an unsupervised way the possible branches of differentiation.

## 3.1 Calculate diffusion 

In [ ]:
sc.pl.draw_graph(adata,color=["MKI67","CDC20","DLX6-AS1","NEUROD6"])

In [ ]:
pca_projections = pd.DataFrame(adata.obsm['X_pca'], index=adata.obs_names)
dm_res = palantir.utils.run_diffusion_maps(pca_projections, n_components=5)

In [ ]:
ms_data = palantir.utils.determine_multiscale_space(dm_res)

In [ ]:
adata.obsm.keys()

In [ ]:
dm_res

In [ ]:
draw_graph = pd.DataFrame(adata.obsm['X_draw_graph_fa'], index=adata.obs_names)

for eigV in dm_res["EigenVectors"]:
    adata.obs['EigenVector_'+str(eigV)] = dm_res["EigenVectors"][eigV]


In [ ]:
sc.pl.draw_graph(adata,color=["EigenVector_0","EigenVector_1","EigenVector_2","EigenVector_3","EigenVector_4"],size=120)

In [ ]:
start_cell = adata.obs_names[adata.obs['root']==1][0]

In [ ]:
pr_res = palantir.core.run_palantir(ms_data, start_cell, num_waypoints=50)

In [ ]:
draw_graph.columns=["x","y"]

In [ ]:
palantir.plot.plot_palantir_results(pr_res, draw_graph)

In [ ]:
adata.obs['palan_pseudoT'] = pr_res.pseudotime
adata.obs['palan_entropy'] = pr_res.entropy
for i, branch in enumerate(pr_res.branch_probs.columns):
    adata.obs['branch_'+branch] = pr_res.branch_probs.loc[adata.obs.index, branch]

In [ ]:
adata.obs.columns.to_list()

In [ ]:
import matplotlib as mpl
import matplotlib.font_manager

sc.pl.draw_graph(adata,color=["palan_pseudoT","palan_entropy"])
sc.pl.draw_graph(adata,color=[i for i in adata.obs.columns.to_list() if i.startswith('branch_')])

# 6. Save

## 6.1 Save AData

In [ ]:
adata.write(results_file)

## 6.2 Timestamp finished computations 

In [ ]:
print(datetime.now())

## 6.3 Save python and html versions

In [ ]:
nb_fname = '6_Pseudotime_intermediate'
nb_fname

In [ ]:
%%bash -s "$nb_fname"
jupyter nbconvert "$1".ipynb --to="python"
jupyter nbconvert "$1".ipynb --to="html"